In [2]:
import numpy as np
import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_absolute_error
from encdec_ad_tensorflow.plots import plot
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import os
import math
from sklearn.cluster import KMeans
from sklearn.metrics import precision_score
import seaborn as sns
from sklearn import metrics
import time
import pickle
import metric_learn
from sklearn.decomposition import PCA  
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import LocalOutlierFactor
import functools
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from encdec_ad_tensorflow.model import EncoderDecoder
from sklearn.metrics import fbeta_score
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.python.client import device_lib; print(device_lib.list_local_devices())
import tensorflow as tf
tf.config.list_physical_devices('GPU')
from utils import make_data, make_data_rev, make_label, calc_leq, leq_filter, validate, figure, figure_detail, plot_timeseries, fig_pr, auc_gs, fig_th_f

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4379722071988583916
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7798259712
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15486590616004280461
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18053114016940928543
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7798259712
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5160612226288894781
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [3]:
def time_series_to_sequences(x, L, S):
    # 長さLのシーケンスをSステップ間隔で取り出していく
    # 取り出したシーケンス(特徴量1の場合)の形状は2次元で shape=(L, 1)
    # tf.expand_dims(〇, axis=0)により次元を増やして，〇の形状をshape=(1, L, 1)に変形
    # tf.concat([], axis=0)で取り出したシーケンスを縦方向に結合して，データセットshape=(None, L, 1)を作る
    return tf.concat([tf.expand_dims(x[i - L: i], axis=0) for i in range(L, len(x) + S, S)], axis=0)

def sequences_to_time_series(x):
    # Transform the sequences back to time series.
    # 部分時系列に変換する際にステップをウィンドウサイズにしている必要がある
    return tf.reshape(x, (x.shape[0] * x.shape[1], x.shape[2]))


def get_anomaly_scores(x, mu, sigma):
    # Calculate the anomaly scores.
    return tf.map_fn(elems=x, fn=lambda x: tf.squeeze(tf.matmul(tf.matmul(x - mu, tf.linalg.inv(sigma)), x - mu, transpose_b=True)))

In [4]:
data_dir='filtered_csv_label'
original = pd.read_csv( os.path.join(data_dir, 'learning_data_10.csv') )
df = pd.read_csv( os.path.join(data_dir, 'learning_data_10.csv') )
train_df=df[(df['day']!=1)&(df['day']<=5)]
test_df=df[df['day']==1]

In [5]:
X_train=train_df[['original']].values
X_test=test_df[['original']].values

## 訓練・検証データ前処理

In [6]:
window_size=10     # Number of time steps.
step_size=1

#  正常な時系列データxn を訓練データと検証データに分ける
# x_sn 訓練データセット
X_train = tf.cast(X_train, dtype=tf.float32)

# 最小値最大値を求めて正規化
x_min = tf.reduce_min(X_train, axis=0, keepdims=True)
x_max = tf.reduce_max(X_train, axis=0, keepdims=True)
# Scale the time series.
X_train_sc = (X_train - x_min) / (x_max - x_min)

# スライディングウィンドウ
x_train = time_series_to_sequences(X_train_sc, L=window_size, S=step_size).numpy()

# 訓練データと検証データに分ける
x_train, x_valid = train_test_split(x_train, test_size=0.3, random_state=0)
x_train = tf.cast(x_train, dtype=tf.float32)
x_valid = tf.cast(x_valid, dtype=tf.float32)

In [15]:
batch_size=512
train_ds = tf.data.Dataset.from_tensor_slices((x_train, x_train)).shuffle(10000).batch(batch_size)
valid_batch_size=2048  
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, x_valid)).batch(valid_batch_size)

## テストデータ前処理

In [16]:
X_test = tf.cast(X_test, tf.float32)
# Scale the time series.
X_test_sc = (X_test- x_min) / (x_max - x_min)

# Split the time series into sequences
x_test=time_series_to_sequences(X_test_sc, L=window_size, S=step_size)

# テストデータセットを作成し、メモリの都合上バッチに分割
test_batch = 2048  
test_ds = tf.data.Dataset.from_tensor_slices(x_test).batch(test_batch)

## 訓練

In [ ]:
# Build the model.
c=64      # Number of hidden units.
d=0.3     # Dropout rate.
learning_rate=0.001
early_stopping_patience=10
max_epochs=1
verbose=1


model = EncoderDecoder(
    L=window_size,  # input length
    m=X_train.shape[-1],  # Number of time series.
    c=c,  # hidden size
    d=d   # dropout rate
)

# Train the model.
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.MeanSquaredError()
)

callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=early_stopping_patience,
    restore_best_weights=True
)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(
    train_ds,
    epochs=max_epochs,
    batch_size=128,
    validation_data=valid_ds,
    callbacks=[tensorboard_callback],
    verbose=verbose
)

In [ ]:
# バッチ処理で予測を行う
predictions = []
for test_x in test_ds:
    batch_predictions = model(test_x, training=False)
    predictions.extend(batch_predictions.numpy().tolist())

predictions=np.array(predictions)
error= tf.keras.losses.mae(predictions.reshape(predictions.shape[0], predictions.shape[1]), 
                       x_test.numpy().reshape(x_test.shape[0], x_test.shape[1])).numpy()

In [ ]:
anomary_score=(error-error.min())/(error.max()-error.min())
# visualize the error curve
plot_timeseries([X_test_sc[:-(window_size-1)].numpy(), predictions[:, 0]], 
                test_df['label'].values[:-(window_size-1)], anomary_score)